In [1]:
import pandas as pd
import numpy as np
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
from matplotlib.widgets import Cursor, Slider, Button, RadioButtons
import matplotlib.cm as cm # matplotlib's color map library

In [2]:
data = pd.read_csv('12032020a.txt', sep='\t', header=None)

In [3]:
data[0].iloc[0] = 'x'
data[1].iloc[0] = 'y'
data.columns = data.iloc[0]
data = data.drop(0).reset_index(drop='True')

C:\Users\sass856\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [4]:
data.head()

,x,y,-228.0,-227.0,-226.0,-225.0,-224.0,-222.0,-221.0,-220.0,...,1715.0,1716.0,1717.0,1718.0,1718.0,1719.0,1720.0,1721.0,1722.0,1722.0
0,0,0,1603.0,1600.0,1601.0,1602.0,1606.0,1604.0,1599.0,1602.0,...,1604.0,1603.0,1603.0,1603.0,1603.0,1599.0,1600.0,1602.0,1607.0,1604.0
1,0,0.002,1603.0,1600.0,1601.0,1600.0,1603.0,1606.0,1604.0,1603.0,...,1617.0,1611.0,1614.0,1614.0,1614.0,1616.0,1615.0,1615.0,1616.0,1613.0
2,0,0.004,1605.0,1598.0,1604.0,1601.0,1601.0,1605.0,1602.0,1602.0,...,1616.0,1614.0,1609.0,1613.0,1617.0,1617.0,1611.0,1612.0,1611.0,1619.0
3,0,0.006,1602.0,1605.0,1600.0,1603.0,1600.0,1596.0,1598.0,1602.0,...,1611.0,1615.0,1614.0,1613.0,1613.0,1611.0,1612.0,1612.0,1617.0,1609.0
4,0,0.008,1606.0,1608.0,1603.0,1606.0,1604.0,1601.0,1600.0,1601.0,...,1614.0,1620.0,1615.0,1612.0,1621.0,1615.0,1615.0,1615.0,1614.0,1615.0


In [5]:
data.shape

(5776, 2002)

In [6]:
# values is wavenumber to plot
Z = data.pivot_table(index='x', columns='y', values=260).T.values

X_unique = np.sort(data['x'].unique())
Y_unique = np.sort(data['y'].unique())
X, Y = np.meshgrid(X_unique, Y_unique)

In [7]:
dataT = data.copy()
dataT = dataT.drop(['x', 'y'], axis=1)
dataT = dataT.T

In [8]:
%matplotlib tk

wavenumber = [0]
pos=[[0,0]]
position = [[0,0]]

fig, ax_list = plt.subplots(1, 2)
rcParams['figure.figsize'] = 12, 8 # sets plot size

ax_list[0].contourf(X,Y,Z, 15, cmap=cm.Blues)
ax_list[1].plot(dataT.index, dataT[position[-1]], color = 'tab:blue')
ax_list[1].axvline(x=wavenumber[-1], color = 'black', linestyle = 'dashed')
txt = fig.text(.8, .9, wavenumber[-1], fontsize=12)

def onclick(event):
    ix, iy = event.xdata, event.ydata
    global txt
    
    if event.inaxes == ax_list[1]:
        wavenumber.append(round(ix))           
        Z = data.pivot_table(index='x', columns='y', values=wavenumber[-1]).T.values
        #replot
        txt.remove()
        txt = fig.text(.8, .9, wavenumber[-1], fontsize=12)
        ax_list[0].cla()
        ax_list[0].contourf(X,Y,Z, 15, cmap=cm.Blues)
        ax_list[1].cla()
        ax_list[1].plot(dataT.index, dataT[position[-1]], color = 'tab:blue')
        ax_list[1].axvline(x=wavenumber[-1], color = 'black', linestyle = 'dashed')
        plt.draw() #redraw
            
    if event.inaxes == ax_list[0]:
        pos.append([min(X[0], key=lambda x:abs(x-ix)), min(Y.T[0], key=lambda x:abs(x-iy))])
        position.append(data.loc[(data['x'] == pos[-1][0]) & (data['y'] == pos[-1][1])].index[0])
        #replot
        ax_list[1].cla()
        ax_list[1].plot(dataT.index, dataT[position[-1]], color = 'tab:blue')
        ax_list[1].axvline(x=wavenumber[-1], color = 'black', linestyle = 'dashed')
        plt.draw() #redraw

In [9]:
cid = fig.canvas.mpl_connect('button_press_event', onclick)

In [10]:
#fig.canvas.mpl_disconnect(cid)

In [11]:
#line cuts
#spatially resolved spectra
